In [5]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
dataset = r'F:\Projects\Mask Detection\Mask Dataset'
imagePaths = list(paths.list_images(dataset))

In [7]:
data = []
labels = []

for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

In [8]:
data = np.array(data, dtype='float32')
labels = np.array(labels)

In [9]:
data.shape

(1376, 224, 224, 3)

In [10]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

In [11]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [12]:
train_X, test_X, train_Y, test_Y = train_test_split(data, labels, test_size=0.20, random_state=10, stratify=labels)

In [13]:
train_X.shape

(1100, 224, 224, 3)

In [14]:
train_Y.shape

(1100, 2)

In [15]:
test_X.shape

(276, 224, 224, 3)

In [16]:
test_Y.shape

(276, 2)

In [17]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True,vertical_flip=True, fill_mode='nearest')

In [18]:
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

In [19]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [20]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='Flatten')(headModel)
headModel = Dense(128, activation = 'relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation = 'softmax')(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

In [21]:
for layer in baseModel.layers:
    layer.trainable = False

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [23]:
learning_rate = 0.0001
Epochs = 20
BS = 12

opt = Adam(learning_rate = learning_rate, decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(
    aug.flow(train_X, train_Y, batch_size=BS),
    steps_per_epoch = len(train_X) // BS,
    validation_data = (test_X, test_Y),
    validation_steps = len(test_X) // BS,
    epochs = Epochs
)

model.save(r"F:\Projects\Mask Detection\mobilenet_v2.model")

Epoch 1/20
91/91 [==============================] - 27s 265ms/step - loss: 0.4874 - accuracy: 0.8134 - val_loss: 0.2152 - val_accuracy: 0.9674
Epoch 2/20
91/91 [==============================] - 26s 282ms/step - loss: 0.2526 - accuracy: 0.9357 - val_loss: 0.1097 - val_accuracy: 0.9855
Epoch 3/20
91/91 [==============================] - 25s 271ms/step - loss: 0.1611 - accuracy: 0.9605 - val_loss: 0.0772 - val_accuracy: 0.9855
Epoch 4/20
91/91 [==============================] - 25s 269ms/step - loss: 0.1151 - accuracy: 0.9752 - val_loss: 0.0642 - val_accuracy: 0.9855
Epoch 5/20
91/91 [==============================] - 25s 269ms/step - loss: 0.0936 - accuracy: 0.9789 - val_loss: 0.0570 - val_accuracy: 0.9855
Epoch 6/20
91/91 [==============================] - 25s 269ms/step - loss: 0.0781 - accuracy: 0.9779 - val_loss: 0.0525 - val_accuracy: 0.9855
Epoch 7/20
91/91 [==============================] - 25s 270ms/step - loss: 0.0726 - accuracy: 0.9825 - val_loss: 0.0511 - val_accuracy: 0.9855

INFO:tensorflow:Assets written to: F:\Projects\Mask Detection\mobilenet_v2.model\assets


INFO:tensorflow:Assets written to: F:\Projects\Mask Detection\mobilenet_v2.model\assets


In [24]:
predict = model.predict(test_X, batch_size = BS)
predict = np.argmax(predict, axis = 1)
print(classification_report(test_Y.argmax(axis = 1), predict, target_names = lb.classes_))

23/23 [==============================] - 5s 167ms/step
              precision    recall  f1-score   support

   with_mask       0.98      0.99      0.98       138
without_mask       0.99      0.98      0.98       138

    accuracy                           0.98       276
   macro avg       0.98      0.98      0.98       276
weighted avg       0.98      0.98      0.98       276

